### CM1 organization gradient standard output

In [1]:
import sys
import os
import numpy as np
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import metpy.calc as mpc
from metpy.units import units
# for animations
from PIL import Image

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# aggregation indices
os.chdir('/data2/willytsai/python_module')
import conorgidx_revise as agi
from SCAI_calc4obj import labeled_obj, SCAI_calc
from COP_calc4obj import COP
from MICA_calc4obj import MICA

In [4]:
def running_mean(y,window_N):
    y_avg = np.zeros(len(y))
    avg_mask = np.ones(window_N) / window_N

    y_avg = np.convolve(y, avg_mask, 'same')
    y_avg[-window_N:] = y[-window_N:]
    y_avg[:window_N] = y[:window_N]
    return y_avg    

In [5]:
def read_data(data_dir,t_start=-144):
    os.chdir(data_dir)
    print(data_dir)
    file_name = glob('*nc')
    data_zon = xr.open_dataset('cm1out_zonmean.nc')
    data_3d = xr.open_dataset('cm1out_sub3d.nc')
    data_2d = xr.open_dataset('cm1out_2Dvars.nc')
    
    # 3d data
    th3d = data_3d.th[t_start:,:30,:,:]
    w3d = data_3d.winterp[t_start:,:,:,:]
    
    # zonal mean data
    th = data_zon.th[t_start:,:,:,:]
    qv = data_zon.qv[t_start:,:,:,:]
    u = data_zon.uinterp[t_start:,:,:,:]
    v = data_zon.vinterp[t_start:,:,:,:]
    w = data_zon.winterp[t_start:,:,:,:]
    qc = data_zon.qc[t_start:,:,:,:]
    qi = data_zon.qi[t_start:,:,:,:]
    prs = data_zon.prs[t_start:,:,:,:]
    # 2d data
    prate = data_2d.prate[t_start:,:,:]
    pwat = data_2d.pwat[t_start:,:,:]
    cwp = data_2d.cwp[t_start:,:,:]
    cape = data_2d.cape[t_start:,:,:]
    cin = data_2d.cin[t_start:,:,:]
    
    # temperature conversion
    T = th/((1000*100/prs)**(287.15/1004))-273.15 # [degC]
    # density 
    ro = prs/(287.15*(T+273.15))

    x_dim = data_2d.lon
    y_dim = data_2d.lat
    div = np.gradient(v,y_dim*1000,axis=2) # du/dx+dv/dy 
    vor = -np.gradient(u,y_dim*1000,axis=2) # -du/dy+dv/dx
    #relative humidty calculation, Buck (1996)
    es_liq = 0.61121*np.exp((18.678-T/234.5)*(T/(257.14+T)))*10 #[hpa]
    es_ice = 0.61115*np.exp((23.036-T/333.7)*(T/(279.82+T)))*10

    qs_liq = 0.622*es_liq/(prs/100-es_liq)
    qs_ice = 0.622*es_ice/(prs/100-es_ice)

    rh = qv/qs_liq
    rh_ice = qv/qs_ice

    rh = rh.values.flatten()
    rh_ice = rh_ice.values.flatten()
    T_test = T.values.flatten()

    rh[T_test<0] = rh_ice[T_test<0]
    rh = rh.reshape((T.shape[0],T.shape[1],T.shape[2],T.shape[3]))
    rh = xr.DataArray(rh,coords=[T.time,T.lev,T.lat,T.lon],dims=['time','lev','lat','lon'])
    
    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u,v,w,div,vor,cape,cin,prs,ro,th3d,w3d)

In [6]:
# get weather or weather2 
#server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/'
server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne'
os.chdir(server)

exp_name = ['CM1_RAD4K_MPI_SFXint',
            'CM1_RAD4K_RCEP_240x240_lowU05',
            'CM1_RAD4K_RCEP_240x240_lowU07',
            'CM1_RAD4K_RCEP_240x240_lowU10',
            'CM1_RAD4K_RCEP_240x240_lowU12',
            'CM1_RAD4K_RCEP_240x240_lowU15']

print(exp_name)
print('Number of Exp: ', len(exp_name))

['CM1_RAD4K_MPI_SFXint', 'CM1_RAD4K_RCEP_240x240_lowU05', 'CM1_RAD4K_RCEP_240x240_lowU07', 'CM1_RAD4K_RCEP_240x240_lowU10', 'CM1_RAD4K_RCEP_240x240_lowU12', 'CM1_RAD4K_RCEP_240x240_lowU15']
Number of Exp:  6


In [7]:
exp_label = ['CTRL','U05','U07','U10','U12','U15']

In [8]:
(th1,T1,qv1,qc1,qi1,rh1,prate1,pwat1,cwp1,u1,v1,w1,div1,vor1,cape1,cin1,prs1,ro1,th1_3d,w1_3d) = read_data(server+'/'+exp_name[0])
(th2,T2,qv2,qc2,qi2,rh2,prate2,pwat2,cwp2,u2,v2,w2,div2,vor2,cape2,cin2,prs2,ro2,th2_3d,w2_3d) = read_data(server+'/'+exp_name[1])
(th3,T3,qv3,qc3,qi3,rh3,prate3,pwat3,cwp3,u3,v3,w3,div3,vor3,cape3,cin3,prs3,ro3,th3_3d,w3_3d) = read_data(server+'/'+exp_name[2])
(th4,T4,qv4,qc4,qi4,rh4,prate4,pwat4,cwp4,u4,v4,w4,div4,vor4,cape4,cin4,prs4,ro4,th4_3d,w4_3d) = read_data(server+'/'+exp_name[3])
(th5,T5,qv5,qc5,qi5,rh5,prate5,pwat5,cwp5,u5,v5,w5,div5,vor5,cape5,cin5,prs5,ro5,th5_3d,w5_3d) = read_data(server+'/'+exp_name[4])
(th6,T6,qv6,qc6,qi6,rh6,prate6,pwat6,cwp6,u6,v6,w6,div6,vor6,cape6,cin6,prs6,ro6,th6_3d,w6_3d) = read_data(server+'/'+exp_name[5])

/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_RAD4K_MPI_SFXint
/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_RAD4K_RCEP_240x240_lowU05
/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_RAD4K_RCEP_240x240_lowU07
/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_RAD4K_RCEP_240x240_lowU10
/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_RAD4K_RCEP_240x240_lowU12
/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_RAD4K_RCEP_240x240_lowU15


In [9]:
x_dim = pwat1.lon
y_dim = pwat1.lat
z_dim = T1.lev
t_dim = np.arange(len(T1.time))/3

In [10]:
fig_out = '/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/fig_sum/'

In [ ]:
color_label=['k','r','g','b']

### ranked vertical velocity 

In [ ]:
def ranked_omega(ro,w_3d):
    omega = -9.8*ro[0,:,0,0]*w_3d
    vint_omega = np.trapz(omega[7:33,:,:,:],z_dim[7:33]*1000,axis=0)
    
    # convective updrafts (omega) in shear and non-shear regions
    vint_omega_s = vint_omega[:,65:151,:] 
    omega_s = omega[:,:,65:151,:]
    vint_omega_ns = np.concatenate((vint_omega[:,-43:,:],vint_omega[:,:43,:]),axis=1)
    omega_ns = np.concatenate((omega[:,:,-43:,:],omega[:,:,:43,:]),axis=2)

    vint_omega_1ds = vint_omega_s[:,:,:].reshape(360*vint_omega_s.shape[1]*vint_omega_s.shape[2])
    vint_omega_1dns = vint_omega_ns[-360:,:,:].reshape(360*vint_omega_ns.shape[1]*vint_omega_ns.shape[2])
    omega_1ds = omega_s[:,-360:,:,:].values.reshape(65,360*omega_s.shape[2]*omega_s.shape[3])
    omega_1dns = omega_ns[:,-360:,:,:].reshape(65,360*omega_ns.shape[2]*omega_ns.shape[3])
    
    # sorting index
    idx_sorted = np.argsort(vint_omega_1ds)
    p_90 = int(0.1*len(idx_sorted)) # return idx 
    p_99 = int(0.01*len(idx_sorted)) # return idx 
    p_999 = int(0.001*len(idx_sorted)) # return idx 
    
    omega_mean = omega_1ds[:,:].mean(axis=1)
    updft_p90 = omega_1ds[:,idx_sorted[:p_90]].mean(axis=1)
    updft_p99 = omega_1ds[:,idx_sorted[:p_99]].mean(axis=1)
    updft_p999 = omega_1ds[:,idx_sorted[:p_999]].mean(axis=1)
    
    dndft_p90 = omega_1ds[:,idx_sorted[-p_90:]].mean(axis=1)
    dndft_p99 = omega_1ds[:,idx_sorted[-p_99:]].mean(axis=1)
    dndft_p999 = omega_1ds[:,idx_sorted[-p_999:]].mean(axis=1)
    
    return omega_mean, updft_p90, updft_p99, updft_p999, dndft_p90, dndft_p99, dndft_p999

In [ ]:
%%time
# calculating for all cases
omega_mean = np.empty((len(exp_label),len(z_dim)))
updft_p90 = np.copy(omega_mean)
updft_p99 = np.copy(omega_mean)
updft_p999 = np.copy(omega_mean)
dndft_p90 = np.copy(omega_mean)
dndft_p99 = np.copy(omega_mean)
dndft_p999 = np.copy(omega_mean)

for n,(ro,w_3d) in enumerate(zip([ro1,ro2,ro3,ro4],[w1_3d,w2_3d,w3_3d,w4_3d])):
    (omega_mean[n,:],updft_p90[n,:],updft_p99[n,:],updft_p999[n,:], 
                     dndft_p90[n,:],dndft_p99[n,:],dndft_p999[n,:]) = ranked_omega(ro,w_3d)

In [ ]:
# sorting omega
fig,ax = plt.subplots(1,3,figsize=(15,4))

for n in range(len(exp_label)):
    ax[0].plot(omega_mean[n,:],z_dim,color=color_label[n])
    ax[1].plot(updft_p90[n,:],z_dim,color=color_label[n])
    ax[2].plot(updft_p99[n,:],z_dim,color=color_label[n])
    
    ax[1].plot(dndft_p90[n,:],z_dim,'--',color=color_label[n])
    ax[2].plot(dndft_p99[n,:],z_dim,'--',color=color_label[n])
    
ax[0].set_ylim([0,18]);#plt.xlim([0,10])
ax[1].set_ylim([0,18])
ax[2].set_ylim([0,18])

In [ ]:
%%time
#omega_p90, omega_p99, omega_p999 = ranked_omega(ro1,w1_3d)
plt.plot(omega_p90,z_dim,'k')
plt.plot(omega_p99,z_dim,'b')
plt.plot(omega_p999,z_dim,'r')
plt.ylim([0,18]);plt.xlim([0,10])